In [ ]:
from fitizens_libraries.load_data import load_labeled_data
from scipy.signal import find_peaks
from Autolabeler import workout
import pandas as pd
import os
import plotly.express as px
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score
import joblib 
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
import warnings
import numpy as np
from sklearn.preprocessing import StandardScaler

In [ ]:
folder_path = "LABELED"
os.makedirs(folder_path, exist_ok=True) #Referenciamos la carpeta LABELED en la que están las carpetas zip con los json
#Ahora voy a iterar en esa carpeta LABELED para obtener la ruta de los archivos, que es el LABELED/NOMBRE y eso lo guardo en una lista
file_names = []
for name in os.listdir(folder_path):
    file_path = f"{folder_path}/{name}"
    file_names.append(file_path)
#Ahora tengo que especificar mis features 
signals = ["accX", "accY", "accZ", "gyroX", "gyroY", "gyroZ", "magnX", "magnY", "magnZ", "linAccX", "linAccY", "linAccZ"]

In [ ]:
data = workout(filelist=file_names,
                       signals= signals)

In [ ]:
df= pd.concat(data)

In [ ]:
df.head()

In [ ]:
import numpy as np
np.unique(df.exercise,return_counts=True)

In [ ]:
import seaborn as sns
sns.countplot(x=df['exercise'], label = "squad")

In [ ]:
df.columns

In [ ]:
df.loc[df['exercise'] == 'NO_EXERCISE','exercising_periods'].unique()

In [ ]:
import plotly.express as px
prim_1000 = df.head(2000)
fig = px.line(prim_1000, x=prim_1000.index, y='linAccZ', color="exercise",title='Time serie of exercise linAccZ')
fig.show()

In [ ]:
prim_100 = df.head(167)
fig = px.line(prim_100, x=prim_100.index, y='linAccZ', color="exercise",title='Time serie of exercise linAccZ')
fig.show()

In [ ]:
siguientes_170 = df.iloc[168:336]

fig = px.line(siguientes_170, x=siguientes_170.index, y='linAccZ', color="exercise", title='Time serie of exercise linAccZ')
fig.show()

In [ ]:
siguientes_170 = df.iloc[336:510]

fig = px.line(siguientes_170, x=siguientes_170.index, y='linAccZ', color="exercise", title='Time serie of exercise linAccZ')
fig.show()

In [ ]:
modelo = joblib.load('modeloXGB.pkl')

In [ ]:
X = df.drop(['peaks', 'first_sample','last_sample', 'first_sample_closest_peak', 
             'last_sample_closest_peak','exercise','accY', 'exercising_periods'], axis=1)
y = df['exercise']

In [ ]:
X.columns

In [ ]:
X

## Modelo XGBOOST

In [ ]:
tamano_ventana = 170 
nuevo_df = pd.DataFrame()
nuevo_df_predicciones = pd.DataFrame()
features = ['mean_accX', 'mean_accZ', 'mean_linAccZ', 'mean_accZ_mod',
       'mean_gyroX_mod', 'mean_linAccX_mod', 'std_accZ', 'std_gyroX', 'std_gyroZ',
       'std_magnX', 'std_linAccX', 'std_accZ_mod', 'std_linAccX_mod',
       'median_accX', 'median_accZ', 'median_linAccZ', 'median_accZ_mod',
       'median_gyroX_mod', 'median_linAccX_mod']
label_encoder = LabelEncoder()

for i in range(0, len(X) - tamano_ventana + 1, tamano_ventana):
    warnings.filterwarnings("ignore")
    # Seleccionar la ventana de datos
    ventana = X.iloc[i:i + tamano_ventana]
    #print(ventana)

    # Calcular estadísticas (promedio, desviación estándar, mediana) para cada columna de la ventana
    promedio_serie = ventana.mean()
    std_serie = ventana.std()
    median_serie = ventana.median()
    nuevo_df = nuevo_df.append(pd.concat([promedio_serie.add_prefix('mean_'), std_serie.add_prefix('std_'), median_serie.add_prefix('median_')]), ignore_index=True)

    
    X1= nuevo_df[features].copy()
    y_pred = modelo.predict(X1)
    X1['start'] = ventana.index[0]
    X1['end'] = ventana.index[-1]
    X1['prediccion'] = y_pred[0]
    nuevo_df_predicciones = pd.concat([nuevo_df_predicciones, X1], ignore_index=True)

In [ ]:
nuevo_df_predicciones.head()

In [ ]:
np.unique(nuevo_df_predicciones.prediccion,return_counts=True)

## Modelo de Naive Bayes

In [ ]:
tamano_ventana = 170 
nuevo_df = pd.DataFrame()
nuevo_df_predicciones = pd.DataFrame()
features = ['mean_accX', 'mean_accZ', 'mean_linAccZ', 'mean_accZ_mod',
       'mean_gyroX_mod', 'mean_linAccX_mod', 'std_accZ', 'std_gyroX', 'std_gyroZ',
       'std_magnX', 'std_linAccX', 'std_accZ_mod', 'std_linAccX_mod',
       'median_accX', 'median_accZ', 'median_linAccZ', 'median_accZ_mod',
       'median_gyroX_mod', 'median_linAccX_mod']
# Scale Data
scaler =  joblib.load('scaler3.pkl')
modelo2 = joblib.load('modeloNB.pkl')

for i in range(0, len(X) - tamano_ventana + 1, tamano_ventana):
    warnings.filterwarnings("ignore")
    # Seleccionar la ventana de datos
    ventana = X.iloc[i:i + tamano_ventana]
    #print(ventana)

    # Calcular estadísticas (promedio, desviación estándar, mediana) para cada columna de la ventana
    promedio_serie = ventana.mean()
    std_serie = ventana.std()
    median_serie = ventana.median()
    nuevo_df = nuevo_df.append(pd.concat([promedio_serie.add_prefix('mean_'), std_serie.add_prefix('std_'), median_serie.add_prefix('median_')]), ignore_index=True)

    
    X1= nuevo_df[features].copy()
    X1_full = scaler.transform(X1)
    y_pred = modelo2.predict(X1_full)
    X1['start'] = ventana.index[0]
    X1['end'] = ventana.index[-1]
    X1['prediccion'] = y_pred[0]
    nuevo_df_predicciones = pd.concat([nuevo_df_predicciones, X1], ignore_index=True)

In [ ]:
np.unique(nuevo_df_predicciones.prediccion,return_counts=True)

In [ ]:
nuevo_df_predicciones

## Modelo Regresion logistica

In [ ]:
tamano_ventana = 170 
nuevo_df = pd.DataFrame()
nuevo_df_predicciones = pd.DataFrame()
features = ['mean_accX', 'mean_accZ', 'mean_linAccZ', 'mean_accZ_mod',
       'mean_gyroX_mod', 'mean_linAccX_mod', 'std_accZ', 'std_gyroX', 'std_gyroZ',
       'std_magnX', 'std_linAccX', 'std_accZ_mod', 'std_linAccX_mod',
       'median_accX', 'median_accZ', 'median_linAccZ', 'median_accZ_mod',
       'median_gyroX_mod', 'median_linAccX_mod']
# Scale Data
scaler = StandardScaler()
modelo3 = joblib.load('modeloLR.pkl')

for i in range(0, len(X) - tamano_ventana + 1, tamano_ventana):
    warnings.filterwarnings("ignore")
    # Seleccionar la ventana de datos
    ventana = X.iloc[i:i + tamano_ventana]
    #print(ventana)

    # Calcular estadísticas (promedio, desviación estándar, mediana) para cada columna de la ventana
    promedio_serie = ventana.mean()
    std_serie = ventana.std()
    median_serie = ventana.median()
    nuevo_df = nuevo_df.append(pd.concat([promedio_serie.add_prefix('mean_'), std_serie.add_prefix('std_'), median_serie.add_prefix('median_')]), ignore_index=True)

    
    X1= nuevo_df[features].copy()
    X1_full = scaler.transform(X1)
    y_pred = modelo3.predict(X1_full)
    X1['start'] = ventana.index[0]
    X1['end'] = ventana.index[-1]
    X1['prediccion'] = y_pred[0]
    nuevo_df_predicciones = pd.concat([nuevo_df_predicciones, X1], ignore_index=True)

In [ ]:
np.unique(nuevo_df_predicciones.prediccion,return_counts=True)

In [ ]:
nuevo_df_predicciones.head()

In [ ]:
len(std_serie)

In [ ]:
f"Predicciones para la ventana {i+1}-{i+tamano_ventana}"

In [ ]:
print(classification_report(y,y_pred))

In [ ]:
# Realizar la búsqueda del overlap óptimo
tamanos_ventana = 170  # Puedes ajustar este rango según tus necesidades

# Incluir el tamaño de ventana deseado en la lista
for tamano_ventana in tamanos_ventana:
    for overlap in porcentajes_overlap:
        # Calcular el tamaño de paso (stride) basado en el porcentaje de overlap
        paso = int(tamano_ventana * (1 - overlap / 100))

        # Inicializar la validación cruzada en series temporales
        tscv = TimeSeriesSplit(n_splits=(len(df) - tamano_ventana) // paso)
        window = X_test.iloc[i:i+tamano_ventana]
        # Almacenar las puntuaciones de precisión para cada configuración
        puntuaciones = []

        for train_index, test_index in tscv.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            
            window_features = []

            # Hacer predicciones usando el modelo cargado
            y_pred = modelo.predict(X_test)

            # Calcular la precisión
            precision = accuracy_score(y_test, y_pred)
            puntuaciones.append(precision)

        # Almacenar la precisión promedio para esta configuración
        clave_configuracion = (tamano_ventana, overlap)
        resultados[clave_configuracion] = sum(puntuaciones) / len(puntuaciones)

# Imprimir los resultados
for configuracion, precision in resultados.items():
    print(f"Configuración: Tamaño de ventana {configuracion[0]} datos, Overlap {configuracion[1]}%, Precisión promedio: {precision}")